# Clustering Crypto

In [149]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [150]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path, index_col=0)
df_crypto.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [151]:
# Keep all the cryptocurrencies that are being traded.
df_crypto_trading = df_crypto[df_crypto.IsTrading == True]
df_crypto_trading.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [152]:
# Remove the "IsTrading" column. 
df_crypto_dropTrading = df_crypto_trading.drop('IsTrading', axis='columns')
df_crypto_dropTrading.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [153]:
# Remove rows that have at least 1 null value.
df_crypto_dropTrading.dropna(inplace=True)
df_crypto_dropTrading.count()

CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64

In [154]:
# Keep the rows where coins are mined.
df_crypto_coins = df_crypto_dropTrading[df_crypto_dropTrading.TotalCoinsMined > 0]
df_crypto_coins.count()

CoinName           532
Algorithm          532
ProofType          532
TotalCoinsMined    532
TotalCoinSupply    532
dtype: int64

In [155]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_crypto_names = df_crypto_coins.filter(items=['CoinName'])
df_crypto_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [156]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df_crypto_dropCoinName = df_crypto_coins.drop('CoinName', axis='columns')
df_crypto_dropCoinName.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [157]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(df_crypto_dropCoinName, columns=['Algorithm', 'ProofType'])
X.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
# Standardize the data with StandardScaler().
df_crypto_scaled = StandardScaler().fit_transform(X)
print(df_crypto_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [159]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(df_crypto_scaled)
print(crypto_pca)

[[-0.33405396  1.03100468 -0.4789349 ]
 [-0.31738675  1.03110476 -0.47925879]
 [ 2.3094393   1.65321553 -0.60004976]
 ...
 [ 0.32309853 -2.34740307  0.39118509]
 [-0.1491682  -2.03824777  0.38542813]
 [-0.29143083  0.7912169  -0.21288366]]


In [160]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC1", "PC2", "PC3"], index=df_crypto_dropCoinName.index)
pcs_df.head(10)

,PC1,PC2,PC3
42,-0.334054,1.031005,-0.478935
404,-0.317387,1.031105,-0.479259
1337,2.309439,1.653216,-0.600050
BTC,-0.151250,-1.321603,0.118347
ETH,-0.157918,-2.051212,0.279493
LTC,-0.165932,-1.130549,0.005562
DASH,-0.396951,1.247853,-0.455292
XMR,-0.157822,-2.211863,0.421924
ETC,-0.156360,-2.051318,0.279484
ZEC,-0.149167,-2.038248,0.385428


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [161]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k":k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [162]:
# Initialize the K-Means model.
model = KMeans(n_clusters=3, random_state=5)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [163]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = df_crypto_dropCoinName.join(pcs_df, how='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(df_crypto_names, how='inner')

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334054,1.031005,-0.478935,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317387,1.031105,-0.479259,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309439,1.653216,-0.600050,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.151250,-1.321603,0.118347,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.157918,-2.051212,0.279493,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.165932,-1.130549,0.005562,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.396951,1.247853,-0.455292,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.157822,-2.211863,0.421924,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.156360,-2.051318,0.279484,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.149167,-2.038248,0.385428,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [164]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC1", y="PC2", z="PC3", color="Class", symbol="Class", width=800, hover_name="CoinName", hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [165]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [166]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df.CoinName.count()} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [167]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
data = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
scaler = MinMaxScaler()
clustered_scaled_df = scaler.fit_transform(data)
print(clustered_scaled_df)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [169]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=clustered_scaled_df, columns=['TotalCoinsMined', 'TotalCoinSupply'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class']

plot_df.head(10)

,TotalCoinsMined,TotalCoinSupply,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [171]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols="CoinName")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)